In [1]:
import os
import sys
from pathlib import Path
from itertools import product, chain
from multiprocessing import Pool
from Bio import SeqIO

sys.path.append('../src')
from utils import run_cmd

# by genome depth

In [2]:
asm = '/media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/20211028_R18-1656H/reference.fa'
raw = '/media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/20211028_R18-1656H/reads.fastq.gz'
save_dir = Path('/media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/_20211028_R18-1656H_/fastq')
save_dir.mkdir(exist_ok=True, parents=True)

In [3]:
# gsize = sum(len(record) for record in SeqIO.parse(asm, 'fasta'))
gsize

4916241

In [ ]:
1405679724/gsize

In [4]:
depths = (50, 60, 70, 80, 90, 100)
repeats = (1, 2, 3, 4, 5)

In [5]:
cmds = []
for depth, repeat in product(depths, repeats):
    outfile = save_dir/f"{depth}x-{repeat}.fastq.gz"
    cmd = f"rasusa -i {raw} -g {gsize} -c {depth} | pigz -9 -p 4 > {outfile}"
    cmds.append(cmd)
cmds[:5]

['rasusa -i /media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/20211028_R18-1656H/reads.fastq.gz -g 4916241 -c 50 | pigz -9 -p 4 > /media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/_20211028_R18-1656H_/fastq/50x-1.fastq.gz',
 'rasusa -i /media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/20211028_R18-1656H/reads.fastq.gz -g 4916241 -c 50 | pigz -9 -p 4 > /media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/_20211028_R18-1656H_/fastq/50x-2.fastq.gz',
 'rasusa -i /media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/20211028_R18-1656H/reads.fastq.gz -g 4916241 -c 50 | pigz -9 -p 4 > /media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/_20211028_R18-1656H_/fastq/50x-3.fastq.gz',
 'rasusa -i /media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/20211028_R18-1656H/reads.fastq.gz -g 4916241 -c 50 | pigz -9 -p 4 > /media/GenomicRe

In [6]:
gsize = '5MB'

cmds = []
dirpath = Path('/media/Central_Lab_Storage/MinION/20210414_Bacteria/fastq/nanopore')
for i in (6, 7):
    infile = dirpath/f"barcode{i:02}.fastq.gz"
    outfile = dirpath/f"barcode{i:02}_50x.fastq.gz"
    cmd = f"rasusa -i {infile} -g {gsize} -c 50 | pigz -9 -p 8 > {outfile}"
    cmds.append(cmd)
cmds

['rasusa -i /media/Central_Lab_Storage/MinION/20210414_Bacteria/fastq/nanopore/barcode06.fastq.gz -g 5MB -c 50 | pigz -9 -p 8 > /media/Central_Lab_Storage/MinION/20210414_Bacteria/fastq/nanopore/barcode06_50x.fastq.gz',
 'rasusa -i /media/Central_Lab_Storage/MinION/20210414_Bacteria/fastq/nanopore/barcode07.fastq.gz -g 5MB -c 50 | pigz -9 -p 8 > /media/Central_Lab_Storage/MinION/20210414_Bacteria/fastq/nanopore/barcode07_50x.fastq.gz']

In [7]:
with Pool(8) as p:
    try:
        p.map(run_cmd, cmds)
        p.close()
        p.join()
    except KeyboardInterrupt:
        p.terminate()

# by data size

In [ ]:
raw = '/media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/20211007_R18-1656/reads.fastq.gz'
outdir = Path('/media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/amplification/20211007_R18-1656/classify/subsample_by_bases')
for i in range(1, 11):
    keep_bases = f"{i**2}MB"
    outfile = outdir/f'{keep_bases}.fastq'
    run_cmd(f"rasusa -i {raw} -b {keep_bases} -o {outfile}")

In [ ]:
keep_bases = 165000000
dirpath = Path('/media/Central_Lab_Storage/MinION/20210901_clBC38_CRE0019/fastq/nanopore')
outpath = Path('/media/Central_Lab_Storage/MinION/20210901_clBC38_CRE0019/fastq/subsamplies/')

for i in dirpath.iterdir():
    outfile = outpath/i.name
    cmd = f"rasusa -i {i} -b {keep_bases} | pigz -9 -p 16 > {outfile}"
    run_cmd(cmd)